# Kafka -> Spark -> Cassandra 파이프라인 스크립트

### 1. 라이브러리 import 및 SparkSession 생성

In [7]:
import findspark
findspark.init("/usr/local/lib/spark-3.3.2-bin-hadoop3")

from pyspark import SparkConf
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import *
from pyspark.sql.functions import udf, col, from_json, pandas_udf, split

session = SparkSession.builder \
    .appName("Jupyter_Notebook") \
    .master("yarn") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.2,com.datastax.spark:spark-cassandra-connector_2.12:3.3.0") \
    .getOrCreate()

23/03/23 02:00:47 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
23/03/23 02:00:57 WARN Client: Same path resource file:///root/.ivy2/jars/org.apache.spark_spark-sql-kafka-0-10_2.12-3.3.2.jar added multiple times to distributed cache.
23/03/23 02:00:57 WARN Client: Same path resource file:///root/.ivy2/jars/com.datastax.spark_spark-cassandra-connector_2.12-3.3.0.jar added multiple times to distributed cache.
23/03/23 02:00:57 WARN Client: Same path resource file:///root/.ivy2/jars/org.apache.spark_spark-token-provider-kafka-0-10_2.12-3.3.2.jar added multiple times to distributed cache.
23/03/23 02:00:57 WARN Client: Same path resource file:///root/.ivy2/jars/org.apache.kafka_kafka-clients-2.8.1.jar added multiple times to distributed cache.
23/03/23 02:00:57 WARN Client: Same path resource file:///root/.ivy2/jars/org.apache.commons_commons-pool2-2.11.1.jar added multiple times to distributed cache.
23/03/23 02:0

### 2. Kafak에서 binary 형태로 받아오는 객체의 스키마 정의, JSON 포맷으로 구조화 

In [8]:
kafka_bootstrap_servers = 'master01:9092,master02:9092,slave01:9092,slave02:9092,slave03:9092'
topic = 'tagmanager'
schema = StructType(
        [
                StructField("serviceToken", StringType()),
                StructField("clientId", LongType()),
                StructField("serviceId", LongType()),
                StructField("sessionId", StringType()),
                StructField("event", StringType()),
                StructField("targetId", StringType()),
                StructField("positionX", IntegerType()),
                StructField("positionY", IntegerType()),
                StructField("location", StringType()),
                StructField("prevLocation", StringType()),
                StructField("referrer", StringType()),
                StructField("timestamp", LongType()),
                StructField("pageDuration", LongType())
        ]
)

streaming_df = session \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
  .option("key.deserializer", "org.apache.kafka.common.serialization.StringDeserializer") \
  .option("value.deserializer", "org.apache.kafka.common.serialization.StringDeserializer") \
  .option("failOnDataLoss","False") \
  .option("subscribe", topic) \
  .load() \
  .withColumn("key", col("key").cast("string")) \
  .withColumn("value", from_json(col("value").cast("string"), schema))
streaming_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: struct (nullable = true)
 |    |-- serviceToken: string (nullable = true)
 |    |-- clientId: long (nullable = true)
 |    |-- serviceId: long (nullable = true)
 |    |-- sessionId: string (nullable = true)
 |    |-- event: string (nullable = true)
 |    |-- targetId: string (nullable = true)
 |    |-- positionX: integer (nullable = true)
 |    |-- positionY: integer (nullable = true)
 |    |-- location: string (nullable = true)
 |    |-- prevLocation: string (nullable = true)
 |    |-- referrer: string (nullable = true)
 |    |-- timestamp: long (nullable = true)
 |    |-- pageDuration: long (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



### 3. Kafka에서 받아온 데이터를 Cassandra 테이블의 형태로 변환

In [9]:
streaming_query = streaming_df.select("key", "value.*") \
    .withColumnRenamed("serviceToken", "service_token") \
    .withColumnRenamed("clientId", "client_id") \
    .withColumnRenamed("serviceId", "service_id") \
    .withColumnRenamed("sessionId", "session_id") \
    .withColumnRenamed("event", "event") \
    .withColumnRenamed("targetId", "target_id") \
    .withColumnRenamed("positionX", "position_x") \
    .withColumnRenamed("positionY", "position_y") \
    .withColumnRenamed("location", "location") \
    .withColumnRenamed("prevLocation", "prev_location") \
    .withColumnRenamed("referrer", "referrer") \
    .withColumnRenamed("timestamp", "creation_timestamp") \
    .withColumnRenamed("pageDuration", "page_duration")
streaming_query.printSchema()

root
 |-- key: string (nullable = true)
 |-- service_token: string (nullable = true)
 |-- client_id: long (nullable = true)
 |-- service_id: long (nullable = true)
 |-- session_id: string (nullable = true)
 |-- event: string (nullable = true)
 |-- target_id: string (nullable = true)
 |-- position_x: integer (nullable = true)
 |-- position_y: integer (nullable = true)
 |-- location: string (nullable = true)
 |-- prev_location: string (nullable = true)
 |-- referrer: string (nullable = true)
 |-- creation_timestamp: long (nullable = true)
 |-- page_duration: long (nullable = true)



### 4. Cassandra sink로 데이터 저장

```
CREATE TABLE stream (
key TEXT,
service_token TEXT,
client_id BIGINT,
service_id BIGINT,
session_id TEXT,
event TEXT,
target_id TEXT,
position_x INT,
position_y INT,
location TEXT,
prev_location TEXT,
referrer TEXT,
creation_timestamp TIMESTAMP,
page_duration BIGINT,
PRIMARY KEY ((service_id), creation_timestamp, session_id)
) WITH CLUSTERING ORDER BY (creation_timestamp DESC);
```

In [ ]:
cassandra_keyspace = "tagmanager"
cassandra_table = "stream"

query = streaming_query.writeStream.outputMode("append") \
      .format("org.apache.spark.sql.cassandra") \
  .option("checkpointLocation", "/") \
  .option("spark.cassandra.connection.host", "master01") \
  .option("spark.cassandra.connection.port", 9042) \
  .option("keyspace", cassandra_keyspace) \
  .option("table", cassandra_table) \
  .option("spark.cassandra.connection.remoteConnectionsPerExecutor", 10) \
  .option("spark.cassandra.output.concurrent.writes", 1000) \
  .option("spark.cassandra.concurrent.reads", 512) \
  .option("spark.cassandra.output.batch.grouping.buffer.size", 1000) \
  .option("spark.cassandra.connection.keep_alive_ms", 600000000) \
      .start()
query.awaitTermination()

23/03/23 08:34:32 WARN DeprecatedConfigParameter: spark.cassandra.connection.keep_alive_ms is deprecated (DSE 6.0.0) and has been automatically replaced with parameter spark.cassandra.connection.keepAliveMS. 
23/03/23 08:34:35 WARN DeprecatedConfigParameter: spark.cassandra.connection.keep_alive_ms is deprecated (DSE 6.0.0) and has been automatically replaced with parameter spark.cassandra.connection.keepAliveMS. 
23/03/23 08:34:35 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/03/23 08:34:36 WARN DeprecatedConfigParameter: spark.cassandra.connection.keep_alive_ms is deprecated (DSE 6.0.0) and has been automatically replaced with parameter spark.cassandra.connection.keepAliveMS. 
23/03/23 08:34:36 WARN DeprecatedConfigParameter: spark.cassandra.connection.keep_alive_ms is deprecated (DSE 6.0.0) and has been automatically replaced with parameter spark.cassandra.connection.keepAliveMS. 
23/03/23 08:34:36 WA

23/03/23 08:34:51 WARN DeprecatedConfigParameter: spark.cassandra.connection.keep_alive_ms is deprecated (DSE 6.0.0) and has been automatically replaced with parameter spark.cassandra.connection.keepAliveMS. 
23/03/23 08:34:51 WARN DeprecatedConfigParameter: spark.cassandra.connection.keep_alive_ms is deprecated (DSE 6.0.0) and has been automatically replaced with parameter spark.cassandra.connection.keepAliveMS. 
23/03/23 08:34:51 WARN DeprecatedConfigParameter: spark.cassandra.connection.keep_alive_ms is deprecated (DSE 6.0.0) and has been automatically replaced with parameter spark.cassandra.connection.keepAliveMS. 


23/03/23 08:35:23 WARN DeprecatedConfigParameter: spark.cassandra.connection.keep_alive_ms is deprecated (DSE 6.0.0) and has been automatically replaced with parameter spark.cassandra.connection.keepAliveMS. 
23/03/23 08:35:23 WARN DeprecatedConfigParameter: spark.cassandra.connection.keep_alive_ms is deprecated (DSE 6.0.0) and has been automatically replaced with parameter spark.cassandra.connection.keepAliveMS. 
23/03/23 08:35:23 WARN DeprecatedConfigParameter: spark.cassandra.connection.keep_alive_ms is deprecated (DSE 6.0.0) and has been automatically replaced with parameter spark.cassandra.connection.keepAliveMS. 
23/03/23 08:35:23 WARN DeprecatedConfigParameter: spark.cassandra.connection.keep_alive_ms is deprecated (DSE 6.0.0) and has been automatically replaced with parameter spark.cassandra.connection.keepAliveMS. 
23/03/23 08:35:23 WARN DeprecatedConfigParameter: spark.cassandra.connection.keep_alive_ms is deprecated (DSE 6.0.0) and has been automatically replaced with parame

In [6]:
session.stop()